In [13]:
import pandas as pd
import numpy as np
import _pickle as pkl
from sklearn.model_selection import train_test_split
import re
import os
from bs4 import BeautifulSoup
from collections import Counter
from string import punctuation
from scipy.sparse import hstack
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.stem.lancaster import LancasterStemmer
from nltk import pos_tag, word_tokenize

# Test and Train

In [3]:
df = pd.read_csv('./datasets/train.csv')
X = df.loc[:, 'Page content'].to_numpy()
y = df.loc[:,'Popularity'].to_numpy()

### Tokenizer

In [7]:
POSITIVE_WORDS = os.path.join(os.getcwd(), 'datasets', 'positive-words.txt')
NEGATIVE_WORDS = os.path.join(os.getcwd(), 'datasets', 'negative-words.txt')
pos_words = []
neg_words = []


for line in open(POSITIVE_WORDS, 'r').readlines()[35:]:
    word = line.rstrip()
    pos_words.append(word)

for line in open(NEGATIVE_WORDS, 'r').readlines()[35:]:
    word = line.rstrip()
    neg_words.append(word)

In [8]:
stop = stopwords.words('english')

def tokenizer_stem_neg(text):
    stemmer = LancasterStemmer()
    return [stemmer.stem(w) for w in re.split(r'\s+', text.strip()) \
            if w in neg_words and re.match('[a-zA-Z]+', w)]

def tokenizer_stem_pos(text):
    stemmer = LancasterStemmer()
    return [stemmer.stem(w) for w in re.split(r'\s+', text.strip()) \
            if w in pos_words and re.match('[a-zA-Z]+', w)]
def tokenizer_stem_nostop(text):
    porter = PorterStemmer()
    return [porter.stem(w) for w in re.split(r'\s+', text.strip()) \
            if w not in stop and re.match('[a-zA-Z]+', w)]

### Out of mem 

In [ ]:
def get_stream(path, size):
    for chunk in pd.read_csv(path, chunksize=size):
        yield chunk
batch_size = 1024
iters = int((27643+batch_size-1)/(batch_size*2))

In [ ]:
stream = get_stream(path='./datasets/train.csv', size=batch_size)
idx = 0
X_train, y_train = None, None
batch_X, batch_y = None, None
for z in range(iters):
    batch = next(stream)
    if(idx==0):
        X_train= batch['Page content']
        y_train = batch['Popularity']
        idx+=1
    else:
        X_train = np.concatenate((X_train, batch['Page content']))
        y_train = np.concatenate((y_train, batch['Popularity']))
    batch = next(stream)

### Features Selection


In [17]:
def feature_selection_part1(data):
    feature_str = ""
    soup = BeautifulSoup(data, 'html.parser')

    # Title
    feature_str += soup.find("h1", {"class": "title"}).text + " "

    # # Channel
    # feature_str += soup.find("article").get("data-channel") + " "

    # # Related Topics
    # feature_str += soup.find("footer", {"class": 'article-topics'}).text.replace(" Topics: ", "").replace(",", "")
    
    section = soup.find("section", {"class": "article-content"})
    paragraph = section.find_all("p")
    first = 0
    last =""
    last_2=""
    for tag in paragraph:
        if(not first):
            feature_str += tag.text + " "
            first = 1
        else:
            last_2=last
            last = tag.text
    if(last_2):feature_str += last_2 + " "
    feature_str = re.sub(r'[.:\']', '', feature_str.lower())
    
    return feature_str
print(feature_selection_part1(X[0]))

nasas grand challenge stop asteroids from destroying earth world asteroid asteroids challenge earth space us world there may be killer asteroids headed for earth, and nasa has decided to do something about it the space agency announced a new "grand challenge" on june 18 to find all dangerous space rocks and figure out how to stop them from destroying our planet see also how it works nasa asteroid-captureresponses to the request for information, which also seeks ideas for detecting and mitigating asteroid threats, are due july 18the asteroid-retrieval mission, designed to provide the first deep-space mission for astronauts flying on nasas space launch system rocket and orion space capsule under development, has come under fire from lawmakers who would prefer that nasa return to the moona draft nasa authorization bill from the house space subcommittee, which is currently in debate, would cancel the mission and steer the agency toward other projects that bill will be discussed during a he

In [12]:
import math
def feature_selection_part2(data, istraining):
    X = []
    idx=0
    author_score =dict()
    author_num = dict()
    channel_score = dict()
    channel_num = dict()
    topic_score = dict()
    topic_num = dict()
    avg_author = 0
    avg_channel = 0
    avg_topic = 0
    if(istraining):
        for html in data:
            soup = BeautifulSoup(html, 'html.parser')
            
            channel = soup.find("article").get("data-channel")
            if channel in channel_score:
                channel_score[channel] += 1 if int(y[idx])==1 else  0
                channel_num[channel] += 1
            else : 
                channel_score[channel] = 0
                channel_score[channel] += 1 if int(y[idx])==1 else  0
                channel_num[channel] = 1
            
            topics = soup.find("footer", {"class": 'article-topics'}).text.replace(" Topics: ", "").split(",")
            for topic in topics:
                if topic in topic_score:
                    topic_score[topic] += 1 if int(y[idx])==1 else  0
                    topic_num[topic] += 1
                else : 
                    topic_score[topic] = 0
                    topic_score[topic] += 1 if int(y[idx])==1 else  0
                    topic_num[topic] = 1


            # Author
            author_re = r'(?:By\s|by\s)?([a-zA-Z]+(\s[A-Z][a-z]+)*)'
            if soup.head.find("span") == None:
                continue
            else:
                author = re.search(author_re, soup.head.find("span").text).group(1)
                if author in author_score:
                    author_score[author] += 1 if int(y[idx])==1 else  0
                    author_num[author] += 1
                else : 
                    author_score[author] = 0
                    author_num[author] = 1
                    author_score[author] += 1 if int(y[idx])==1 else  0
            idx+=1
        total_channel = 0
        total_topic = 0
        total_author = 0
        for html in data:
            for c,s in channel_score.items():
                avg_channel+=s
                total_channel+=channel_num[c]
            avg_channel = avg_channel/total_channel

            for c,s in topic_score.items():
                avg_topic+=s
                total_topic+=topic_num[c]
            avg_topic = avg_topic/total_topic

            for c,s in author_score.items():
                avg_author+=s
                total_author+=author_num[c]
            avg_author = avg_author/total_author

    for html in data:
        soup = BeautifulSoup(html, 'html.parser')
        feature_list = []
        # Author score
        author_re = r'(?:By\s|by\s)?([a-zA-Z]+(\s[A-Z][a-z]+)*)'
        if soup.head.find("span") == None:
            feature_list.append(avg_author)
        else:
            author = re.search(author_re, soup.head.find("span").text).group(1)
            if author in author_score:
                feature_list.append(author_score[author]/author_num[author])
            else:
                feature_list.append(avg_author)
        # Channel score
        channel = soup.find("article").get("data-channel")
        if channel in channel_score:
            feature_list.append(channel_score[channel]/channel_num[channel])
        else:
            feature_list.append(avg_channel)
        # Topic score
        topics = soup.find("footer", {"class": 'article-topics'}).text.replace(" Topics: ", "").split(",")
        total_score = 0
        order = 0 
        order_denominator = 0
        for i in range(len(topics)):
            order_denominator += math.exp(-0.5*i)

        for topic in topics:
            order_coef = math.exp(-0.5*order)/order_denominator
            if topic in topic_score:
                total_score+=(topic_score[topic]/topic_num[topic])*order_coef
            else:
                total_score+=avg_topic*order_coef
            order+=1
        
        feature_list.append(total_score) 

        # word count of title
        feature_list.append(len(soup.find("h1", {"class": "title"}).text))
        # average word length and unique word rate
        words = re.findall(r'\w+', soup.get_text().lower())
        if words:
            total_words = len(words)
            unique_words = set(words)
            unique_word_count = len(unique_words)
            total_length = sum(len(word) for word in words)
            unique_rate = unique_word_count / total_words
            average_length = total_length / len(words)
            feature_list.append(unique_rate)
            feature_list.append(average_length)
        
        # Time
        if soup.time.text == None or soup.time.text == "":
            feature_list.append(0)
            feature_list.append(0)
        else:
            month = int(re.search(r'(\d+-\d+-\d+)', soup.time.text).group(1).split("-")[1])
            feature_list.append(month)
            hour = int(re.search(r'(\d+:\d+:\d+)', soup.time.text).group(1)[:5].split(":")[0])
            minute = int(re.search(r'(\d+:\d+:\d+)', soup.time.text).group(1)[:5].split(":")[1])
            feature_list.append(hour * 60 + minute)

        # Weekend
        if soup.time.text == None or soup.time.text == "":
            feature_list.append(0)
        else:
            feature_list.append(1 if soup.time.get("datetime")[:3] in ["Sat", "Sun"] else 0)

        # Word Count
        text_p = (''.join(s.findAll(string=True))for s in soup.findAll('p'))
        c_p = Counter((x.rstrip(punctuation).lower() for y in text_p for x in y.split()))
        text_div = (''.join(s.findAll(string=True))for s in soup.findAll('div'))
        c_div = Counter((x.rstrip(punctuation).lower() for y in text_div for x in y.split()))
        total = c_div + c_p
        feature_list.append(len(list(total.elements())))

        section = soup.find("section", {"class": "article-content"})

        # Video + Image count
        img_count = len(section.find_all("img")) + len(section.find_all("picture")) + len(section.find_all("figure"))
        video_count = len(section.find_all("video")) + len(section.find_all("iframe"))
        media_count = img_count + video_count
        feature_list.append(media_count)

        # Appealing count
        link_count = len(section.find_all("a"))
        strong_count = len(section.find_all("strong"))
        appealing_count = link_count + strong_count
        feature_list.append(appealing_count)

        # POS & NEG count
        paragraph = section.find_all("p")
        pos_count = 0
        neg_count = 0
        q_count = 0
        ex_count = 0
        for tag in paragraph:
            pos_count += len(tokenizer_stem_pos(tag.text))
            neg_count += len(tokenizer_stem_neg(tag.text))
            if tag.text.find("?") != -1:
                q_count += tag.text.find("?")
            if tag.text.find("!") != -1:
                ex_count += tag.text.find("!")
        feature_list.append(pos_count)    
        feature_list.append(neg_count)
        feature_list.append(q_count)
        feature_list.append(ex_count)

        X.append(feature_list)
    return np.array(X)
feature_selection_part2(X[0:10],True)

0.04415096306457571 0.04045008071497035 0.04045008071497035
0.0
0.04415096306457571 0.04045008071497035 0.04045008071497035
0.9999999999999999
0.04415096306457571 0.04045008071497035 0.04045008071497035
0.698112354712604
0.04415096306457571 0.04045008071497035 0.04045008071497035
0.13800217235329681
0.04415096306457571 0.04045008071497035 0.04045008071497035
0.33359334091039083
0.04415096306457571 0.04045008071497035 0.04045008071497035
0.01625851413453984
0.04415096306457571 0.04045008071497035 0.04045008071497035
0.8481078423872517
0.04415096306457571 0.04045008071497035 0.04045008071497035
0.0931618616129238
0.04415096306457571 0.04045008071497035 0.04045008071497035
0.9238332397905892
0.04415096306457571 0.04045008071497035 0.04045008071497035
0.0


array([[0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 6.00000000e+01,
        4.89731438e-01, 4.81042654e+00, 6.00000000e+00, 9.04000000e+02,
        0.00000000e+00, 5.43000000e+02, 0.00000000e+00, 1.80000000e+01,
        1.40000000e+01, 3.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [1.00000000e+00, 1.00000000e+00, 1.00000000e+00, 7.40000000e+01,
        4.74719101e-01, 4.65449438e+00, 3.00000000e+00, 1.06000000e+03,
        0.00000000e+00, 3.21000000e+02, 0.00000000e+00, 9.00000000e+00,
        5.00000000e+00, 3.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [5.00000000e-01, 3.33333333e-01, 6.98112355e-01, 6.80000000e+01,
        4.98765432e-01, 4.40987654e+00, 5.00000000e+00, 1.15500000e+03,
        0.00000000e+00, 2.55800000e+03, 5.00000000e+01, 4.00000000e+00,
        3.30000000e+01, 2.10000000e+01, 7.05000000e+02, 3.80000000e+01],
       [5.00000000e-01, 0.00000000e+00, 1.38002172e-01, 4.30000000e+01,
        7.26351351e-01, 4.65540541e+00, 1.00000000e+01, 1.460

In [7]:
def feature_selection_part3(data):
    feature_str = ""
    soup = BeautifulSoup(data, 'html.parser')
    
    
    feature_str = re.sub(r'[.:\',$()`]', '', feature_str.lower())
    return feature_str

print(feature_selection_part3(X[0]))
    

there may be killer asteroids headed for earth and nasa has decided to do something about it the space agency announced a new "grand challenge" on june 18 to find all dangerous space rocks and figure out how to stop them from destroying our planet see also how it works nasa asteroid-captureresponses to the request for information which also seeks ideas for detecting and mitigating asteroid threats are due july 18the asteroid-retrieval mission designed to provide the first deep-space mission for astronauts flying on nasas space launch system rocket and orion space capsule under development has come under fire from lawmakers who would prefer that nasa return to the moona draft nasa authorization bill from the house space subcommittee which is currently in debate would cancel the mission and steer the agency toward other projects that bill will be discussed during a hearing wednesday june 19 at 10 am edtsee also how it works nasa asteroid-capture mission in picturesbut nasa officials defe

In [8]:
from sklearn.linear_model import SGDClassifier
from xgboost import XGBClassifier
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import GradientBoostingClassifier

### Random + GBC

#### training vectorizer

In [9]:
X_train, X_valid, Y_train, Y_valid = train_test_split(X, y, test_size=.2)
print('Begin vectorizer Training...')

tfidf = TfidfVectorizer(ngram_range=(1,2),
                        preprocessor=feature_selection_part1,
                        tokenizer=tokenizer_stem_nostop)
tfidf.fit(X)

# tfidf_type = TfidfVectorizer(ngram_range=(1,2),
#                         preprocessor=feature_selection_part3,
#                         tokenizer=tokenizer_stem_nostop)
# tfidf_type.fit(X)

Begin vectorizer Training...


c:\Users\johnson\.conda\envs\DL2\Lib\site-packages\sklearn\feature_extraction\text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


TfidfVectorizer(ngram_range=(1, 2),
                preprocessor=<function feature_selection_part3 at 0x0000024FC62FDE40>,
                tokenizer=<function tokenizer_stem_nostop at 0x0000024FC97AC220>)

#### training model

In [10]:
# print('Begin Type Training...')
# type_model = SGDClassifier(loss='log_loss', max_iter=200)
# x_train_type = tfidf_type.transform(X_train)
# y_train = LabelEncoder().fit_transform(Y_train)
# type_model.fit(x_train_type, y_train)

print('Begin Word Training...')
word_model = RandomForestClassifier(n_estimator=100, max_features=0.4, max_depth=5, min_samples_leaf=5)
x_train_word = tfidf.transform(X_train)
y_train = LabelEncoder().fit_transform(Y_train)
word_model.fit(x_train_word, y_train)

print('Begin Stats Training...')
stats_model = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0, max_depth=1, random_state=0)
x_train_stats = feature_selection_part2(X_train, True)
stats_model.fit(x_train_stats, y_train)

Begin Type Training...
Begin Word Training...
Begin Stats Training...
-75
-28
62
1
-3
8
6
2
0
3
-15
-50
3
23
32
-29
13
3
-9
3
1
0
-32
16
0
23
2
-20
62
-10
-29
-29
7
62
2
-29
-2
7
32
6
4
1
32
1
6
-48
6
-12
3
-20
-32
1
-2
23
-75
8
-12
-48
-6
-4
22
-29
-1
-75
4
-2
-8
-9
1
1
6
62
8
-9
0
22
-29
-29
-50
-9
3
-50
1
-7
15
-3
62
-14
1
-29
0
3
-29
0
-50
-20
-16
-14
-2
-9
7
62
-75
-14
3
-2
-20
-20
-50
1
32
-5
-32
8
-75
-2
8
-50
4
3
-10
7
-9
-3
-9
-9
-3
1
-7
6
5
4
6
-10
-20
0
4
-5
-50
62
5
6
4
-75
-75
-29
-9
-1
1
-29
-50
8
-12
-6
6
-7
-1
-2
-3
4
-6
-75
-15
11
-7
12
-20
-7
-29
-29
-2
23
-14
-4
-16
11
4
-4
-16
8
-11
-20
-9
-10
62
-9
0
11
3
-5
-12
62
-75
-75
-14
-10
-29
32
-4
-75
-14
1
5
-32
4
-4
0
2
-6
-75
-75
-10
-7
1
-28
-10
2
4
8
0
0
62
4
-50
-2
-16
22
3
-20
23
32
6
11
-3
-9
62
-10
7
62
-1
8
7
6
-9
-48
-9
-32
3
-75
-12
3
3
-75
6
0
2
7
-1
23
22
-12
-29
-1
-10
-1
3
-29
22
14
0
3
3
0
4
1
9
22
-20
1
4
-16
0
3
-7
-1
1
-4
62
4
6
7
-11
-29
-12
-10
7
0
0
-50
-1
-29
-29
-12
-8
3
-20
6
1
-12
0
3
-9
-29
-4


GradientBoostingClassifier(learning_rate=1.0, max_depth=1, random_state=0)

#### evaluation

In [11]:
print('Begin Evaluation...')
y_valid = LabelEncoder().fit_transform(Y_valid)

# y_pred_type_train = type_model.predict_proba(x_train_type)[:,1]
# y_pred_type_valid = type_model.predict_proba(tfidf_type.transform(X_valid))[:,1]

y_pred_word_train = word_model.predict_proba(x_train_word)[:,1]
y_pred_word_valid = word_model.predict_proba(tfidf.transform(X_valid))[:,1]

y_pred_stats_train = stats_model.predict_proba(x_train_stats)[:,1]
y_pred_stats_valid = stats_model.predict_proba(feature_selection_part2(X_valid))[:,1]

# print("Type Model")
# print(f'Train score: {roc_auc_score(y_train, y_pred_type_train)}')
# print(f'Valid score: {roc_auc_score(y_valid, y_pred_type_valid)}')

print("Word Model")
print(f'Train score: {roc_auc_score(y_train, y_pred_word_train)}')
print(f'Valid score: {roc_auc_score(y_valid, y_pred_word_valid)}')

print("Stats Model")
print(f'Train score: {roc_auc_score(y_train, y_pred_stats_train)}')
print(f'Valid score: {roc_auc_score(y_valid, y_pred_stats_valid)}')

Begin Evaluation...
5
-10
1
3
-8
-10
-3
6
1
-7
-7
2
-16
-1
4
4
6
1
-4
-10
-3
-2
-12
1
-2
1
-3
16
-16
12
-14
1
1
-7
-3
-3
5
-3
4
1
1
16
1
0
1
-3
1
20
-4
1
-7
7
10
8
-2
0
-14
-14
-4
-16
-3
-8
-1
-14
-10
-3
1
-1
4
8
-3
6
4
-7
20
-2
-8
4
-8
-3
20
-1
-10
-2
2
1
-1
5
-3
2
5
1
1
1
6
9
-14
-14
12
2
1
16
7
-3
-15
2
-4
1
1
-2
1
16
7
8
-2
-7
20
2
-15
-16
7
-10
7
-5
-1
5
6
8
0
-3
7
-9
5
-1
2
1
7
0
2
-12
-3
9
0
20
1
-9
2
6
1
-10
1
-10
-16
-14
20
16
-4
2
2
0
8
4
-2
-9
-2
2
0
2
2
-2
-3
9
-3
-14
-14
-16
6
1
1
-15
0
-2
7
-7
12
4
-3
1
1
3
-8
16
-2
-7
-9
-2
5
5
-15
-4
2
-15
20
-2
20
-3
-2
20
-2
-7
-3
16
2
7
-2
-14
3
5
0
5
-3
7
-9
-2
5
-2
4
7
-7
-1
-1
1
2
5
12
-10
3
1
2
-8
-8
-10
5
-16
-10
-3
2
-2
-10
16
-1
16
5
-9
1
-2
-14
5
7
0
1
-14
-16
-10
-1
0
4
1
1
-16
1
2
-10
6
-15
-1
2
10
-14
-12
-1
1
10
-16
-1
0
12
-5
-1
-3
12
16
-1
-1
-7
1
1
-3
16
6
-3
1
-3
4
4
-2
-2
-3
4
-3
-5
0
5
-2
2
7
5
2
-3
-10
-3
20
-1
-14
20
8
-10
9
1
-1
1
4
-1
-1
-2
-6
-1
-16
-10
1
-10
2
-15
5
-3
12
10
-10
2
6
-3
-14
-3
-2
2
-3
-15
0
-3


#### find best para

In [12]:
train_score = []
valid_score = []
best_coef = []
best_train = 0
best_valid = 0
coef_ = np.linspace(0,.5,21)
print('Find best para...')
for i in coef_:
    # for j in coef_:
    # train_score = roc_auc_score(y_train, y_pred_word_train*(i) + y_pred_stats_train*(1-i) + y_pred_type_train*(1-i-j))
    # valid_score = roc_auc_score(y_valid, y_pred_word_valid*(i) + y_pred_stats_valid*(1-i) + y_pred_type_valid*(1-i-j))
    train_score = roc_auc_score(y_train, y_pred_word_train*(i) + y_pred_stats_train*(1-i))
    valid_score = roc_auc_score(y_valid, y_pred_word_valid*(i) + y_pred_stats_valid*(1-i))
    if valid_score > best_valid:
        best_valid = valid_score
        best_train = train_score
        best_coef = [i,1-i]
        # best_coef = [i,j,1-i-j]

print(f'Train score: {best_train}')
print(f'Valid score: {best_valid}')

Find best para...


Train score: 0.9647829470148834
Valid score: 0.5757368475761251


### back up model

In [13]:
pkl.dump(word_model, open("output/backup_word.pickle", "wb"))
pkl.dump(stats_model, open("output/backup_stats.pickle", "wb"))
# pkl.dump(type_model, open('output/backup_topic.pickle', "wb"))
pkl.dump(tfidf, open('output/tfidf_part1.pickle', "wb"))
# pkl.dump(tfidf_type, open('output/tfidf_first_paragraph.pickle', "wb"))

### Prediction

In [14]:
df = pd.read_csv('./datasets/test.csv')
print("Content:\n",df.loc[0])

Id = df.loc[:, 'Id'].to_numpy()
X_test = df.loc[:, 'Page content'].to_numpy()
print(X_test.shape)
print("\nId:\n", Id[0])
print("Content:\n", X_test[0])

Content:
 Id                                                          27643
Page content    <html><head><div class="article-info"><span cl...
Name: 0, dtype: object
(11847,)

Id:
 27643
Content:
 <html><head><div class="article-info"><span class="byline "><a href="/author/sam-laird/"><img alt="2016%2f09%2f15%2f63%2fhttpsd2mhye01h4nj2n.cloudfront.netmediazgkymde1lza2.9814b" class="author_image" src="http://i.amz.mshcdn.com/-qaMPB8aiQeIaoBhqlU0OLjA07A=/90x90/2016%2F09%2F15%2F63%2Fhttpsd2mhye01h4nj2n.cloudfront.netmediaZgkyMDE1LzA2.9814b.jpg"/></a><span class="author_name">By <a href="/author/sam-laird/">Sam Laird</a></span><time datetime="Mon, 09 Sep 2013 19:47:02 +0000">2013-09-09 19:47:02 UTC</time></span></div></head><body><h1 class="title">Soccer Star Gets Twitter Death Threats After Tackling One Direction Member</h1><figure class="article-image"></figure><article data-channel="entertainment"><section class="article-content"> <div class="shift-to-hero"> <p><iframe allowfullscreen="" 

In [15]:
# x_test_type = tfidf_type.transform(X_test)
x_test_word = tfidf.transform(X_test)
x_test_stats = feature_selection_part2(X_test)

# y_pred_type_test = type_model.predict_proba(x_test_type)[:,1]
y_pred_word_test = word_model.predict_proba(x_test_word)[:,1]
y_pred_stats_test = stats_model.predict_proba(x_test_stats)[:,1]
y_pred = np.around(y_pred_word_test*(best_coef[0]) + y_pred_stats_test*(best_coef[1]), decimals=3)

13
31
-18
3
1
-7
13
-6
-1
3
-19
1
0
-8
-17
10
26
-1
-54
-17
0
4
-8
-1
-19
3
-17
-1
-1
-8
-8
-7
15
-1
-4
-19
-1
13
10
-14
2
13
1
-3
-13
-18
-5
-4
-8
7
13
-17
-4
3
23
26
-10
3
8
-8
-6
-19
-9
-8
31
1
-8
-54
0
9
3
-1
-1
2
1
15
-1
1
-5
-17
2
-1
-19
1
10
4
23
31
3
-4
20
-12
1
5
20
0
-5
-13
-19
4
31
-9
26
-13
5
-2
13
-9
26
5
-14
3
3
3
15
13
31
-12
10
-1
-4
13
-17
-7
10
1
-18
-18
1
2
23
-17
-5
4
-17
-1
0
5
-1
2
0
5
15
-1
26
1
31
-1
14
3
-18
10
-6
15
3
-6
-4
23
-11
2
-7
4
-6
0
3
5
-1
-6
-8
2
26
7
13
-3
26
3
-9
-1
-3
-6
-6
-1
-7
2
1
-6
23
-14
10
-1
-8
-19
-3
-54
-14
4
-18
-8
11
1
-8
-9
31
-8
26
-8
-18
-4
15
3
-18
5
-54
15
-10
-19
-54
2
-8
-6
5
-5
-17
3
-17
1
-18
3
-18
26
-7
10
15
-1
-17
15
1
2
-17
-14
2
-1
-1
31
15
-8
-18
-1
7
0
-6
-17
3
-17
-16
-13
2
3
-12
-6
13
-10
3
3
5
5
3
-3
-54
1
0
-1
3
3
-18
15
13
-6
-12
11
5
7
-54
-16
31
-8
-1
0
5
26
1
-14
31
1
13
-8
1
-7
-8
3
-54
-3
7
-19
-8
26
4
-16
5
-11
1
13
-17
-8
-54
2
-54
10
-17
3
1
10
26
13
0
14
31
10
-8
26
4
3
0
1
-18
-5
2
3
-19
-7
-17
7
-7
-5
-

In [16]:
output_data = {'Id': Id, 'Popularity': y_pred}
output_dataframe = pd.DataFrame(output_data)
print(output_dataframe)

output_dataframe.to_csv("./datasets/y_pred.csv", index=None)

          Id  Popularity
0      27643       0.463
1      27644       0.621
2      27645       0.423
3      27646       0.447
4      27647       0.495
...      ...         ...
11842  39485       0.525
11843  39486       0.541
11844  39487       0.507
11845  39488       0.635
11846  39489       0.444

[11847 rows x 2 columns]
